In [1]:
import pandas as pd

In [86]:
final_df =pd.read_csv('Divvy Time Series Dataset - Sheet1.csv')

In [87]:
final_df["Year"] = final_df["Year"].fillna(method="ffill")
final_df["Date"] = pd.to_datetime(final_df["Year"].astype(int).astype(str) + "-" + final_df["Month"].astype(str) + "-01")

/var/folders/j2/gbnqt5g13fbdlhczrh40l8780000gn/T/ipykernel_6035/2205364413.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_df["Year"] = final_df["Year"].fillna(method="ffill")


In [95]:
final_df.tail(9)

,Year,Month,Total Counts,Classic Bike Count,Electric Bike Count,Docked Bike Count,Electric Scooter Count,Casual User Count,Member User Count,Date
48,2024.0,4,415025.0,188403.0,226622.0,NaN,NaN,131810.0,283215.0,2024-04-01
49,2024.0,5,609704.0,306915.0,302789.0,NaN,NaN,231150.0,378554.0,2024-05-01
50,2024.0,6,710747.0,337749.0,372998.0,NaN,NaN,301169.0,409578.0,2024-06-01
51,2024.0,7,749004.0,370561.0,378443.0,NaN,NaN,320581.0,428423.0,2024-07-01
52,2024.0,8,755804.0,352997.0,402725.0,NaN,7.0,318398.0,437406.0,2024-08-01
53,2024.0,9,820867.0,314339.0,362273.0,NaN,19900.0,346494.0,474373.0,2024-09-01
54,2024.0,10,616246.0,280597.0,335649.0,NaN,NaN,216429.0,399817.0,2024-10-01
55,2024.0,11,335042.0,148109.0,186933.0,NaN,NaN,93062.0,241980.0,2024-11-01
56,2024.0,12,178359.0,70518.0,107841.0,NaN,NaN,38369.0,139990.0,2024-12-01


In [90]:
may = pd.read_csv('202405-divvy-tripdata.csv')
june = pd.read_csv('202406-divvy-tripdata.csv')
july = pd.read_csv('202407-divvy-tripdata.csv')
august = pd.read_csv('202408-divvy-tripdata.csv')
sep = pd.read_csv('202409-divvy-tripdata.csv')
oct = pd.read_csv('202410-divvy-tripdata.csv')

In [3]:
august.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BAA154388A869E64,classic_bike,2024-08-02 13:35:14.403,2024-08-02 13:48:24.426,State St & Randolph St,TA1305000029,Wabash Ave & 9th St,TA1309000010,41.884621,-87.627834,41.870769,-87.625734,member
1,8752245932EFF67A,electric_bike,2024-08-02 15:33:13.965,2024-08-02 15:55:23.865,Franklin St & Monroe St,TA1309000007,Damen Ave & Cortland St,13133,41.880317,-87.635185,41.915983,-87.677335,member
2,44DDF9F57A9A161F,classic_bike,2024-08-16 15:44:06.233,2024-08-16 15:57:52.109,Franklin St & Monroe St,TA1309000007,Clark St & Elm St,TA1307000039,41.880317,-87.635185,41.902973,-87.631280,member
3,44AAAF069B0C78C3,electric_bike,2024-08-19 18:47:11.855,2024-08-19 18:56:33.269,Clark St & Elm St,TA1307000039,McClurg Ct & Ohio St,TA1306000029,41.902973,-87.631280,41.892592,-87.617289,member
4,77138D500A6B7B4B,classic_bike,2024-08-03 20:34:20.560,2024-08-03 20:46:29.305,Western Ave & Leland Ave,TA1307000140,Ashland Ave & Belle Plaine Ave,13249,41.966400,-87.688704,41.956057,-87.668835,casual


In [94]:
value_counts = sep['member_casual'].value_counts()
print(value_counts)

member_casual
member    474472
casual    346804
Name: count, dtype: int64


In [ ]:
# List of monthly CSV files (update with correct paths if needed)
months = ["may", "june", "july", "august", "sep", "oct"]
file_paths = [f"20240{m}-divvy-tripdata.csv" for m in range(5, 10)] + ["202410-divvy-tripdata.csv"]

Updated final_df with new counts successfully!


In [66]:
monthly_counts = []  # Initialize an empty list before processing CSVs

for file in file_paths:
    df = pd.read_csv(file)

    df["Year"] = pd.to_datetime(df["started_at"]).dt.year
    df["Month"] = pd.to_datetime(df["started_at"]).dt.month

    # Aggregate data
    grouped = df.groupby(["Year", "Month"]).agg(
        **{"Total Counts": ("ride_id", "count"),
        "Casual User Count": ("member_casual", lambda x: (x == "casual").sum()),
        "Member User Count": ("member_casual", lambda x: (x == "member").sum()),
        "Classic Bike Count": ("rideable_type", lambda x: (x == "classic_bike").sum()),
        "Electric Bike Count": ("rideable_type", lambda x: (x == "electric_bike").sum())}
    ).reset_index()

    monthly_counts.append(grouped)  # Append aggregated data

# Combine all months' data into one DataFrame
monthly_counts_df = pd.concat(monthly_counts, ignore_index=True)

monthly_counts_df = monthly_counts_df.groupby(["Year", "Month"], as_index=False).sum()

monthly_counts_df["Date"] = pd.to_datetime(monthly_counts_df["Year"].astype(int).astype(str) + "-" + monthly_counts_df["Month"].astype(str) + "-01")


In [67]:
monthly_counts_df

,Year,Month,Total Counts,Casual User Count,Member User Count,Classic Bike Count,Electric Bike Count,Date
0,2024,5,609704,231150,378554,306915,302789,2024-05-01
1,2024,6,710747,301169,409578,337749,372998,2024-06-01
2,2024,7,749004,320581,428423,370561,378443,2024-07-01
3,2024,8,755804,318398,437406,352997,402725,2024-08-01
4,2024,9,820867,346494,474373,314339,362273,2024-09-01
5,2024,10,616246,216429,399817,280597,335649,2024-10-01


In [88]:
# Merge the two DataFrames on Year and Month
merged_df = final_df.merge(monthly_counts_df, on=["Year", "Month"], how="left", suffixes=("", "_new"))

# Overwrite existing values with new ones (even if they are not NaN)
for col in monthly_counts_df.columns.difference(["Year", "Month"]):  # Exclude key columns
    merged_df[col] = merged_df[col + "_new"].fillna(merged_df[col])  # Only replace if new data exists

# Drop unnecessary columns with "_final" and "_new" suffixes
merged_df = merged_df[final_df.columns]  # Keep original column order

missing_rows = monthly_counts_df[~monthly_counts_df["Date"].isin(merged_df["Date"])]  # Find missing rows

# Step 2: Append missing rows to merged_df
merged_df = pd.concat([merged_df, missing_rows], ignore_index=True)

# Save the updated DataFrame back to final_df
final_df = merged_df

# Sort by Year and Month in ascending order
final_df = final_df.sort_values(by=["Year", "Month"]).reset_index(drop=True)

# Optionally, save it
final_df.to_csv("final_df_updated.csv", index=False)

print("Final_df successfully updated with new values and missing rows added!")

Final_df successfully updated with new values and missing rows added!


In [89]:
final_df

,Year,Month,Total Counts,Classic Bike Count,Electric Bike Count,Docked Bike Count,Electric Scooter Count,Casual User Count,Member User Count,Date
0,2020.0,4,84776.0,NaN,NaN,84776.0,NaN,23628.0,61148.0,2020-04-01
1,2020.0,5,200274.0,NaN,NaN,200274.0,NaN,86909.0,113365.0,2020-05-01
2,2020.0,6,343005.0,NaN,NaN,343005.0,NaN,154718.0,188287.0,2020-06-01
3,2020.0,7,551480.0,NaN,1935.0,549545.0,NaN,269296.0,282184.0,2020-07-01
4,2020.0,8,622361.0,NaN,66195.0,556166.0,NaN,289661.0,332700.0,2020-08-01
5,2020.0,9,532958.0,NaN,128352.0,404606.0,NaN,230692.0,302266.0,2020-09-01
6,2020.0,10,388653.0,NaN,152176.0,236477.0,NaN,145012.0,243641.0,2020-10-01
7,2020.0,11,259716.0,NaN,108134.0,151582.0,NaN,88099.0,171617.0,2020-11-01
8,2020.0,12,131573.0,70616.0,47953.0,13004.0,NaN,30080.0,101493.0,2020-12-01
9,2021.0,1,96834.0,61700.0,33028.0,2106.0,NaN,18117.0,78717.0,2021-01-01
